In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import time
import hashlib, hmac, base64
import requests, json, urllib3

requests.packages.urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import sys

SPINNING_RATE = 0.1
TIME_ADJUST = 0
PRINT_TRACE = 0

# config is passed to most function, this is a json object that includes all the required parameters

def spinning_cursor():
    while True:
        for cursor in '|/-\\':
            yield cursor

def init_params_headers(config, headers, params):
    params.update({'org' : config['org_key']})     
    headers.update({"content-type": "application/json", "Authorization": "Bearer %s" % config['sign'] })


def getTimestamp():
    return str(int(time.time() * 1000) + TIME_ADJUST) ;

def hmacSHA256(api_key, values):
    message = bytes("".join(values),encoding='utf8')
    return base64.b64encode(hmac.new( bytes(api_key, encoding='utf8'), message, digestmod=hashlib.sha256).digest()).decode("utf-8")

def ws_post_sign(config):

    url = "%s/integration/sign" % config['url']
    headers = {"content-type": "application/json"}
    data = { 'uid' : config['user_id'], 'apiKey' : config['api_key']}
    if( PRINT_TRACE) : print("REST CALL: "+url+" "+str(data))
    r = requests.post(url, verify=False, data=json.dumps(data), headers=headers)
    if (r.status_code != 200):
        print("Error: get signature: %s" % r.json()['data'])
        config['sign'] = 0
        return 0
    else:
        config['sign'] = r.json()['sign']
    return r.json()['sign']


def ws_delete_process(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s" % (config['url'], config['project_key'])

    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    r = requests.delete(url, verify=False, params=params, headers=headers)
    if (r.status_code != 200):
        print("Error: delete process: %s" % r.json()['data'])        
    return r

def ws_proc_post(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes" % (config['url'])
    data = { 'title' : config['project_name'], 'org' : config['org_key']}
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(data))
    r = requests.post(url, verify=False, data=json.dumps(data), headers=headers, params=params)
    if (r.status_code != 200):
        print("Error: Process %s creation: %s" % (config['project_name'], r.json()['data']))        
    return r

def ws_csv_upload(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)
    # no content-type 
    headers.pop("content-type")

    url = "%s/integration/csv/%s/upload" % (config['url'], config['project_key'])
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    files = {'file': (config['csv_filename'], open(config['csv_filename'], 'rb'),'text/zip')}
    r = requests.post(url, verify=False, files=files, params=params, headers=headers)
    # Async call --- the caller need to loop on get job status that it is completed
    if (r.status_code != 200):
        print("Error: CSV upload: %s" % r.json()['data'])        
    return r


def ws_query_post(config, query):
    headers = {}
    params = {}
    init_params_headers(config, headers, params)
    headers['content-type'] = 'application/x-www-form-urlencoded'
    url = "%s/analytics/integration/%s/query" % (
        config['url'], config['project_key'])
    data = "params={'query': '%s'}" % query
    print(data)
    r = requests.post(url, verify=False, params=params,
                      headers=headers, data=data)
    if (r.status_code != 200):
        print("Error: query post: %s" % r.json()['data'])
    return r


def ws_backup_upload(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)
    # no content-type 
    headers.pop("content-type")

    url = "%s/integration/processes/%s/upload-backup" % (config['url'], config['project_key'])

    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    files = {'file': (config['backup_filename'], open(config['backup_filename'], 'rb'),'text/zip')}
    r = requests.post(url, verify=False, files=files, params=params, headers=headers)
    if (r.status_code != 200):
        print("Error: backup upload: %s" % r.json()['data']) 
    return r

def ws_get_backup_list(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url  = "%s/integration/processes/%s/backups" % (config['url'], config['project_key'])
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    r = requests.get(url, verify=False, headers=headers, params=params)
    if (r.status_code != 200):
        print("Error: backup list get: %s" % r.json()['data'])
    return r

def getBackupIdByMessage(backuplist, message):
    backuplist = backuplist['backups']
    for backup in backuplist:
        if (backup['message'] == message) :
            return backup['id']
    return 0

def ws_apply_backup(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s/backups/%s" % (config['url'], config['project_key'], config['backup_id'])
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    r = requests.put(url, verify=False, headers=headers, params=params)
    if (r.status_code != 200):
        print("Error: apply backup: %s : %s" % (r.json()['data'], config['backup_id'])) 
    return r

def ws_create_log(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/csv/%s/create-log" % (config['url'], config['project_key'])
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    r = requests.post(url, verify=False, headers=headers, params=params)
    # Async function, the caller needs to check the job status (returned in r)
    if (r.status_code != 200):
        print("Error: create log: %s" % r.json()['data']) 
    return r

def ws_get_csv_job_status(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/csv/job-status/%s" % (config['url'], config['job_key'])
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    r = requests.get(url, verify=False, headers=headers, params=params)
    if (r.status_code != 200):
        print("Error: get CSV job status: %s" % r.json()['data'])        
    return r

def create_and_load_new_project(config):
    print("Process Mining: creating new project")
    r_proc = ws_proc_post(config)
    if (r_proc.status_code != 200): return r_proc
    config['project_key']= r_proc.json()['projectKey']
    print("Process Mining: loading event log (please wait)")
    r = ws_csv_upload(config)
    if (r.status_code != 200): return r
    else :
        # wait until async call is completed
        config['job_key']= r.json()['data']
        runningCall = 1
        spinner = spinning_cursor()
        while  runningCall :
            r = ws_get_csv_job_status(config)
            if (r.json()['data'] == 'complete') :
                runningCall = 0
            if (r.json()['data'] == 'error') :
                runningCall = 0
                print("Error while loading CSV -- column number mismatch")
                return 0
            sys.stdout.write(next(spinner))
            sys.stdout.flush()
            sys.stdout.write('\b')
            time.sleep(SPINNING_RATE)

    r = ws_backup_upload(config)
    if (r.status_code != 200): return r
    config['backup_id'] = r.json()['backupInfo']['id']
    r = ws_apply_backup(config)
    if (r.status_code != 200): return r
    print("Process Mining: refreshing process (please wait)")
    r = ws_create_log(config)
    if (r.status_code != 200): return r
    else :
        # wait until async call is completed
        config['job_key']= r.json()['data']
        runningCall = 1
        #pbar = tqdm(total=100)
        spinner = spinning_cursor()
        while  runningCall :
            r = ws_get_csv_job_status(config)
            if (r.json()['data'] == 'complete') :
                runningCall = 0
            if (r.json()['data'] == 'error') :
                runningCall = 0
                print("Error while creating the log")
                return 0
            sys.stdout.write(next(spinner))
            sys.stdout.flush()
            sys.stdout.write('\b')
            time.sleep(SPINNING_RATE)
    return r_proc

def upload_csv_and_createlog(config) :
    r = ws_csv_upload(config)
    if (r.status_code != 200): return r
    else :
        # wait until async call is completed
        config['job_key']= r.json()['data']
        runningCall = 1
        print("Process Mining: loading event log (please wait)")
        spinner = spinning_cursor()
        while  runningCall :
            r = ws_get_csv_job_status(config)
            if (r.json()['data'] == 'complete') :
                runningCall = 0
            if (r.json()['data'] == 'error') :
                runningCall = 0
                print("Error while loading CSV -- column number mismatch")
                return 0
            sys.stdout.write(next(spinner))
            sys.stdout.flush()
            sys.stdout.write('\b')
            time.sleep(SPINNING_RATE)

    r = ws_create_log(config)
    if (r.status_code != 200): return r
    else :
        # wait until async call is completed
        config['job_key']= r.json()['data']
        runningCall = 1
        print("Process Mining: refreshing model (please wait)")
        spinner = spinning_cursor()
        while  runningCall :
            r = ws_get_csv_job_status(config)
            if (r.json()['data'] == 'complete') :
                runningCall = 0
            if (r.json()['data'] == 'error') :
                runningCall = 0
                print("Error while creating the log")
                return 0
            sys.stdout.write(next(spinner))
            sys.stdout.flush()
            sys.stdout.write('\b')
            time.sleep(SPINNING_RATE)
    return r

# List all the dashboards of a given process mining project
def ws_get_dashboards(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/analytics/integration/dashboard/%s/list" % (config['url'], config['project_key'])
    r = requests.get(url, verify=False, params=params, headers=headers)
    if (r.status_code == 200):
        values = r.json()
        return {'status_code': r.status_code, 'data' : values['data']['dashboards']}
    else:
        return {'status_code': r.status_code, 'data' : None}

# Returns only the table widgets in the dashboard_name
def ws_get_widgets(config):
    res = ws_get_dashboards(config)

    if (res['status_code'] == 200): dashboards = res['data']
    else:
        return {'status_code': res['status_code'], 'data': None}

    for aDashboard in dashboards:
        if (aDashboard['name'] == config['dashboard_name']):
            dashboard = aDashboard

    if dashboard == 0 :
        print("ERROR: dashboard does not exist")
        return 0
    dashboard_id = dashboard['id']

    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/analytics/integration/dashboard/%s/%s/list" % (config['url'], config['project_key'], dashboard_id)
    r = requests.get(url, verify=False, params=params, headers=headers )
    if (res['status_code'] == 200):
        widgets = r.json() 
        return {'status_code': res['status_code'], 'data': widgets['data']['widgets']}
    else:
        return {'status_code': res['status_code'], 'data': None}

def ws_get_widget_values(config):
    dashboard_id = config.get('dashboard_id')
    if dashboard_id is None:
        # add the dashboard_id to the widget data
        print("....searching dashboard_id")
        res = ws_get_dashboards(config)
        if (res['status_code'] == 200): dashboards = res['data']
        else:
            return {'status_code': res['status_code'], 'data': None}
        dashboard = 0
        for aDashboard in dashboards:
            if (aDashboard['name'] == config['dashboard_name']):
                dashboard = aDashboard
        if (dashboard == 0) :
            print("ERROR: dashboard %s does not exist" % config['dashboard_name'])
            return {'status_code': 200, 'data': None}
        else:
            # Store the dashboard id in the widget to avoid calling again the rest API to retrieve the dashboard by name
            config['dashboard_id'] = dashboard['id']

    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/analytics/integration/dashboard/%s/%s/%s/retrieve" % (config['url'],
                                                                        config['project_key'],
                                                                        config['dashboard_id'],
                                                                        config['widget_id'])

    r = requests.get(url, verify=False, params=params, headers=headers )
    if (r.status_code == 200):
        values = r.json()
        return {'status_code': r.status_code, 'data': values['data']}
    else:
        return {'status_code': r.status_code, 'data': None}

def ws_create_update_variables(config, variablesArray):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s/variables" % (config['url'], config['project_key'])
    r = requests.post(url, verify=False, data=json.dumps(variablesArray), params=params, headers=headers )
    return r

def ws_get_variable(config, variablename):
    headers={}
    params={}
    init_params_headers(config, headers, params)
    
    url = "%s/integration/processes/%s/variables/%s" % (config['url'], config['project_key'], variablename)
    r = requests.get(url, verify=False, params=params, headers=headers )
    if (r.status_code == 200):
        values = r.json()
        return values['data']
    else:
        return 0

def ws_get_variables(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s/variables" % (config['url'], config['project_key'])
    r = requests.get(url, verify=False, params=params, headers=headers )
    if (r.status_code == 200):
        values = r.json()
        return values['data']
    else:
        print("Error: ws_get_variables %s" % r.json()['data'])
        return []

def ws_delete_variable(config, variablename):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s/variables/%s" % (config['url'], config['project_key'], variablename)
    r = requests.delete(url, verify=False, params=params, headers=headers )
    return r.status_code

def ws_delete_variables(config):
    variables = ws_get_variables(config)
    for variable in variables:
        ws_delete_variable(config, variable['name'])

def ws_get_processes(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes" % (config['url'])
    r = requests.get(url, verify=False, params=params, headers=headers )
    if (r.status_code == 200):
        values = r.json()
        return values['data']
    else:
        return r

def ws_get_project_info(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s" % (config['url'], config['project_key'])

    return(requests.get(url, verify=False, params=params, headers=headers ))


def ws_get_project_meta_info(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/csv/%s/meta" % (config['url'], config['project_key'])

    r = requests.get(url, verify=False, params=params, headers=headers )
    if (r.status_code == 200):
        values = r.json()
        return values['data']
    else:
        return r

In [ ]:
import random
import json
import time
from datetime import datetime as dt, timedelta
import sys
import ProcessMining_API as IPM
import pandas as pd

def join_and_cleanup(df1, df2, cols, suffix, suffix_for):
    # suffix_for = 'left' or 'right'
    if (suffix_for == 'left'): suffixes = (suffix, '')
    else: suffixes = ('', suffix)
    df = df1.merge(df2, how='left', on=cols, indicator='alert_is_present_on', suffixes=suffixes)
    # remove columns (duplicated) from log
    kept_cols = []
    for col in df.columns:
        if (suffix not in col):
            kept_cols.append(col)
    return df[kept_cols]

class processMiningConnector():
    def __init__(self, config):
        self.config = config

    def copy(self):
        return processMiningConnector(self.config.copy())

class widgetAlerts():
    def __init__(self, connector, dashboardName, widgetId, matchingColumnIndexes): # config is a json with the widget properties
        # create a copy of the connector because connector.config holds the dashboard_id when it is found the first time
        self.connector = connector.copy()
        self.dashboardName = dashboardName
        self.widgetId = widgetId
        self.matchingColumnIndexes = matchingColumnIndexes # replaced when the widget columns are known
        if (self.matchingColumnIndexes == 0 or self.matchingColumnIndexes == ''):
            self.matchingColumnIndexes = [0]
        self.logFileName = 'alert_log_' + self.dashboardName + '_' + self.widgetId +'.csv'
        self.summaryFileName = 'alert_summary_' + self.dashboardName + '_' + self.widgetId +'.csv'
        self.connector.config['dashboard_name'] = dashboardName
        self.connector.config['widget_id'] = widgetId


    def loadLog(self):
        # Load alert log for this widget
        try: 
            self.log_df = pd.read_csv(self.logFileName, dtype=str)
        except:
            # First call
            self.log_df = pd.DataFrame()

    def loadSummary(self):
        # Load summary file for this widget
        try: 
            self.summary_df = pd.read_csv(self.summaryFileName)
        except:
            # First call: Create the summary df 
            self.summary_df = pd.DataFrame()

    def loadWidgetData(self):
        ws_post_sign(self.connector.config)
        res = ws_get_widget_values(self.connector.config)
        if (res['status_code'] == 200):
            self.widget_df = pd.DataFrame(res['data'])
            self.setMatchingColumns() # matching columns: indexes replaced with column names

        else: 
            print('ERROR: Dashboard or Widget not found')
            self.widget_df = pd.DataFrame()

    def setMatchingColumns(self):
        # Get the widget columns used to match the alerts
        # Do the merge with the columns mentionned in the configuration as an array
        self.matchingColumns = []
        for i in range(len(self.widget_df.columns)):
            if (i in self.matchingColumnIndexes):
                self.matchingColumns.append(self.widget_df.columns[i])
        print("Matching columns used for existing alerts: %s" % self.matchingColumns)
    
    def updateAlertsFirstTime(self):
        if len(self.widget_df) == 0:
            print('Empty widget and no log yet. No data to generate')
            self.new_log_df = pd.DataFrame()
            self.summary_df = pd.DataFrame()
        else: #first time execution
            self.new_log_df = self.widget_df.copy()
            self.new_log_df['alert_status'] = 'NEW'
            self.new_log_df['alert_creation_date'] = dt.now().isoformat()
            self.new_log_df['alert_closed_date'] = ''
            self.summary_df = pd.DataFrame([{
                'update_date': dt.now().isoformat(),
                'new': len(self.widget_df),
                'pending': 0,
                'closed': 0,
                'new_to_pending': 0,
                'new_to_closed': 0,
                'pending_to_closed': 0,
                'pending_to_pending': 0,
                'any_to_closed': 0,
                'progression_rate': 0
            }])

    def updateAlerts(self):
        self.loadLog()
        self.loadSummary()
        self.loadWidgetData()

        if len(self.log_df) == 0: # No log yet
            self.updateAlertsFirstTime()
        else:
            self.new_log_df = self.log_df[self.log_df.alert_status == 'CLOSED'] # we don't touch CLOSED alerts
            # Add a new summary row
            self.summary_df.loc[len(self.summary_df)] = { 
                'update_date': dt.now().isoformat(),
                'new': 0,
                'pending': 0,
                'closed': self.summary_df.loc[len(self.summary_df)-1, 'closed'],
                'new_to_pending': 0,
                'new_to_closed': 0,
                'pending_to_closed': 0,
                'pending_to_pending': 0,
                'any_to_closed': 0,
                'progression_rate': 0
            }
            if len(self.widget_df) == 0: # no more alerts, close all NEW and PENDING from log
                self.closeAllAlerts()
            else:
                self.manageNewAlerts()
                self.manageExistingAlerts()
        
        self.log_df = self.new_log_df

    def closeAllAlerts(self):
        new_to_close_df = self.log_df[self.log_df.alert_status == 'NEW']
        new_to_close_df.alert_status = 'CLOSED'
        new_to_close_df.alert_closed_date = dt.now().isoformat()
        pending_to_close_df = self.log_df[self.log_df.alert_status == 'PENDING']
        pending_to_close_df.alert_status = 'CLOSED'
        pending_to_close_df.alert_closed_date = dt.now().isoformat()
        self.new_log_df = pd.concat([self.new_log_df, new_to_close_df, pending_to_close_df])
        self.summary_df.loc[len(self.summary_df) - 1, 'closed'] = len(self.log_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'new_to_closed'] = len(new_to_close_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'pending_to_closed'] = len(pending_to_close_df)

    def manageNewAlerts(self):
        not_closed_df = self.log_df[self.log_df.alert_status != 'CLOSED']
        widget_new_df = join_and_cleanup(self.widget_df, not_closed_df, self.matchingColumns, "_suffixFromlog", 'right')
        # Alerts in the widget with 'exist'==left_only are NEW
        widget_new_df = widget_new_df[widget_new_df.alert_is_present_on == 'left_only']
        widget_new_df.alert_status = 'NEW'
        widget_new_df.alert_creation_date = dt.now().isoformat()
        widget_new_df.alert_closed_date = ''
        self.new_log_df = pd.concat([self.new_log_df, widget_new_df.drop(columns=['alert_is_present_on'])])
        self.summary_df.loc[len(self.summary_df) - 1, 'new'] = len(widget_new_df)

    def manageExistingAlerts(self):
        not_closed_df = self.log_df[self.log_df.alert_status != 'CLOSED']
        not_closed_df = join_and_cleanup(not_closed_df, self.widget_df, self.matchingColumns, "_suffixFromlog", 'left')

        # NEW to PENDING  
        new_to_pending_df = not_closed_df.query('alert_status=="NEW" & alert_is_present_on=="both"')
        self.summary_df.loc[len(self.summary_df) - 1, 'new_to_pending'] = len(new_to_pending_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'pending'] = len(new_to_pending_df)
        if (len(new_to_pending_df)):
            new_to_pending_df.alert_status = 'PENDING'
            self.new_log_df = pd.concat([self.new_log_df, new_to_pending_df.drop(columns=['alert_is_present_on'])])

        # NEW to CLOSED
        new_to_close_df = not_closed_df.query('alert_status=="NEW" & alert_is_present_on=="left_only"')
        self.summary_df.loc[len(self.summary_df) - 1, 'new_to_closed'] = len(new_to_close_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'closed'] += len(new_to_close_df)
        if (len(new_to_close_df)):
            new_to_close_df.alert_status = 'CLOSED'
            new_to_close_df.alert_closed_date = dt.now().isoformat()
            self.new_log_df = pd.concat([self.new_log_df, new_to_close_df.drop(columns=['alert_is_present_on'])])

        # PENDING to PENDING
        pending_to_pending_df = not_closed_df.query('alert_status=="PENDING" & alert_is_present_on=="both"')
        self.summary_df.loc[len(self.summary_df) - 1, 'pending_to_pending'] = len(pending_to_pending_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'pending'] += len(pending_to_pending_df)
        if (len(pending_to_pending_df)):
            pending_to_pending_df.alert_status = 'PENDING'
            self.new_log_df = pd.concat([self.new_log_df, pending_to_pending_df.drop(columns=['alert_is_present_on'])])    

        # PENDING to CLOSED
        pending_to_closed_df = not_closed_df.query('alert_status=="NEW" & alert_is_present_on=="left_only"')
        self.summary_df.loc[len(self.summary_df) - 1, 'pending_to_closed'] = len(pending_to_closed_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'closed'] += len(pending_to_closed_df)
        if (len(pending_to_closed_df)):
            pending_to_closed_df.alert_status = 'CLOSED'
            pending_to_closed_df.alert_closed_date = dt.now().isoformat()
            self.new_log_df = pd.concat([self.new_log_df, pending_to_closed_df.drop(columns=['alert_is_present_on'])])

        self.computeProgressionRate()         

    def saveLogFile(self):
        if (len(self.log_df)):
            self.log_df.to_csv(self.logFileName, index=None)
    
    def saveSummaryFile(self):
        if (len(self.summary_df)):
            self.summary_df.to_csv(self.summaryFileName, index=None)

    def computeProgressionRate(self):
        l = len(self.summary_df)
        if (len(self.summary_df) < 2): # no previous summary
            self.summary_df.loc[l - 1, 'progression_rate'] = 0
        else:
            if (self.summary_df.loc[l - 2, 'pending'] + self.summary_df.loc[l - 2, 'new']):
                # there were pending and/or new alerts last time
                self.summary_df.loc[l - 1, 'progression_rate'] = (self.summary_df.loc[l - 1, 'new_to_closed'] + self.summary_df.loc[l - 1, 'pending_to_closed']) / (self.summary_df.loc[l - 2, 'pending'] + self.summary_df.loc[l - 2, 'new'])
            else: 
                self.summary_df.loc[l - 1, 'progression_rate'] = 0
            
    def updateAlertsAndSave(self):
        self.updateAlerts()
        self.saveLogFile()
        self.saveSummaryFile()

    def setLogFilename(self, filename):
        self.logFileName = filename

    def setSummaryFilename(self, filename):
        self.summaryFileName = filename

    def getLogs(self):
        return self.log_df
    
    def getSummary(self):
        return self.summary_df
    
    def setMatchingColumnIndexes(self, cols):
        self.matchingColumnIndexes = cols

In [ ]:
config_file = './pharoses1_config.json'
try:
    with open(config_file, 'r') as file:
        connectorConfig = json.load(file)
        print('Loading config file')
except:
    connectorConfig = {
        "url":"https://ProcessMining.com",
        "user_id": "john.smith",
        "api_key":"8a5kga87eqvd1180",
        "project_key": "procure-to-pay",
        "org_key": ""
    }
# Constructors
connector = processMiningConnector(connectorConfig)
alerts1 = widgetAlerts(connector, 'alerts', 'invoices-withholding-tax', [0,1,2,3])
# Optionnaly, the default filenames can be changed like this:
alerts1.setLogFilename('mylog.csv')
alerts1.setSummaryFilename('mysummary.csv')
# The matching columns can be changed. Use columns with values that do not change at each update for the same alert (ex: no duration until now)
alerts1.setMatchingColumnIndexes([0,1,2])
alerts1.updateAlertsAndSave() # Load files, get widget values, match with log, update log and summary, save files
print(alerts1.getLogs().head())
print(alerts1.getSummary().head())

alerts2 = widgetAlerts(connector, 'alerts2', 'invoices-blocked-account2', [0,1,2,3])
alerts2.updateAlertsAndSave()
print(alerts2.getLogs().head())
print(alerts2.getSummary().head())
